# ## Análise Exploratória de Dados de Commodities Agrícolas

In [ ]:
# Configuração inicial
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats

plt.style.use('ggplot')
plt.rcParams['figure.figsize'] = (12,6)
pd.options.display.float_format = '{:,.2f}'.format

In [ ]:
# Carregamento dos dados
try:
    data = pd.read_csv('/home/boto-cdr/Documentos/portfolio_projects/food_data/fi.csv', encoding='utf-8')
    print("Dados carregados com sucesso!")
    print(f"Total de registros: {len(data):,}")
except FileNotFoundError:
    print("Erro: Verifique o caminho do arquivo e as permissões!")

## Pré-processamento Básico

In [ ]:
# Verificação e limpeza inicial
print("\nInformações básicas:")
print(data.info())

print("\nValores faltantes por coluna:")
print(data.isnull().sum())

# Tratamento de outliers
if 'Value' in data.columns:
    z_scores = stats.zscore(data[['Value']])
    data_clean = data[(np.abs(z_scores) < 3).all(axis=1)]
    print(f"\nRegistros após remoção de outliers: {len(data_clean):,}")

## Análise Temporal

In [ ]:
# Evolução anual das transações
if 'Year Number' in data_clean.columns:
    plt.figure(figsize=(14,7))
    annual_trend = data_clean.groupby('Year Number')['Value'].sum().reset_index()
    
    sns.lineplot(data=annual_trend, x='Year Number', y='Value')
    plt.title('Evolução Anual do Valor Comercial Total', fontsize=14)
    plt.xlabel('Ano', fontsize=12)
    plt.ylabel('Valor (Milhões USD)', fontsize=12)
    plt.grid(True)
    plt.show()

## Análise Geográfica

In [ ]:
# Top 15 países por volume comercial
if 'Country' in data_clean.columns:
    top_countries = data_clean.groupby('Country')['Value'].sum().nlargest(15)
    
    plt.figure(figsize=(14,8))
    sns.barplot(x=top_countries.values, y=top_countries.index, palette='viridis')
    plt.title('Top 15 Países por Volume Comercial', fontsize=14)
    plt.xlabel('Valor Total (Milhões USD)', fontsize=12)
    plt.ylabel('')
    plt.show()

## Análise de Produtos

In [ ]:
# Distribuição por categoria de commodities
if 'Category' in data_clean.columns:
    category_dist = data_clean['Category'].value_counts().nlargest(10)
    
    plt.figure(figsize=(12,8))
    sns.barplot(x=category_dist.values, y=category_dist.index, palette='rocket')
    plt.title('Distribuição por Categoria de Commodities', fontsize=14)
    plt.xlabel('Número de Transações', fontsize=12)
    plt.ylabel('Categoria', fontsize=12)
    plt.show()

## Análise de Preços

In [ ]:
# Relação valor-quantidade
if {'Value', 'Unit of Measure'}.issubset(data_clean.columns):
    price_quantity = data_clean[data_clean['Unit of Measure'].isin(['Million $', '1,000'])]
    price_quantity = price_quantity.pivot_table(index=['Commodity','Country'], 
                                              columns='Unit of Measure', 
                                              values='Value').reset_index()

    plt.figure(figsize=(14,10))
    sns.scatterplot(data=price_quantity.dropna(), 
                   x='1,000', 
                   y='Million $', 
                   hue='Commodity',
                   size='Million $',
                   sizes=(20, 200))
    plt.title('Relação entre Volume e Valor Comercial', fontsize=14)
    plt.xlabel('Quantidade (1,000 unidades)', fontsize=12)
    plt.ylabel('Valor (Milhões USD)', fontsize=12)
    plt.xscale('log')
    plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left')
    plt.tight_layout()
    plt.show()

## Análise de Concentração de Mercado

In [ ]:
# Índice HHI simplificado
def calculate_hhi(df):
    market_shares = df.groupby('Country')['Value'].sum()
    return (market_shares**2).sum() / (market_shares.sum()**2) * 10000

if 'Country' in data_clean.columns:
    hhi = data_clean.groupby('Year Number').apply(calculate_hhi).reset_index(name='HHI')
    
    plt.figure(figsize=(14,7))
    sns.lineplot(data=hhi, x='Year Number', y='HHI', marker='o')
    plt.axhline(1500, color='red', linestyle='--', label='Mercado Concentrado')
    plt.axhline(2500, color='darkred', linestyle='--', label='Monopólio/Oligopólio')
    plt.title('Índice de Concentração de Mercado (HHI)', fontsize=14)
    plt.ylabel('HHI Score', fontsize=12)
    plt.legend()
    plt.show()

## Análise Setorial Detalhada

In [ ]:
# Top 5 commodities
if 'Commodity' in data_clean.columns:
    top_commodities = data_clean['Commodity'].value_counts().nlargest(5).index
    
    for commodity in top_commodities:
        plt.figure(figsize=(14,6))
        subset = data_clean[data_clean['Commodity'] == commodity]
        sns.lineplot(data=subset, x='Year Number', y='Value', estimator='sum')
        plt.title(f'Evolução de {commodity}', fontsize=14)
        plt.xlabel('Ano', fontsize=12)
        plt.ylabel('Valor (Milhões USD)', fontsize=12)
        plt.show()